In [13]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np 
from pathlib import Path
from bs4 import BeautifulSoup
tqdm.pandas()

path = 'data'

root_folder = Path(path)
root_folder.mkdir(parents=True, exist_ok=True)

In [14]:
import os
os.environ["TIKA_SERVER_JAR"] = r"C:\Users\PC\Documents\tika\tika-server-standard.jar"
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-11\bin"
os.environ["PATH"] += os.pathsep + os.environ["JAVA_HOME"]

import tika
from tika import parser
tika.initVM()

In [15]:
df_searchterms = pd.read_csv(path+'/google_scholar_search_base_article_detail.csv',sep =';')

files = list(root_folder.rglob("*"))  
files = [f for f in files if f.is_file() and f.suffix in (".html",'.pdf')]
lst_name_files = [file.name for file in files]
df_searchterms = df_searchterms[df_searchterms['article_file'].isin(lst_name_files)].copy()
df_searchterms['path_article_file'] = path +'/articles/'+ df_searchterms['folder']+'/'+df_searchterms['article_file']
df_searchterms['path_txt_folder'] = path +'/articles_txt/'+ df_searchterms['folder']
df_searchterms['path_txt_article'] = df_searchterms['path_txt_folder']+'/'+df_searchterms['article_file'].str.replace('.html','.txt').str.replace('.pdf','.txt')
df_searchterms.sort_values(by = ['domain_count','domain','folder'],\
                            ascending=[False,False,False], ignore_index=True, inplace=True)
df_searchterms.drop_duplicates(subset = ['article_file'], keep='first', inplace= True)

In [16]:
df_searchterms.to_csv(path+'/google_scholar_search_base_article_detail_txt.csv', sep = ';', index = False)

In [17]:
records = df_searchterms.to_dict(orient = 'records')

In [18]:
for record in tqdm(records):


    full_path_folder = Path(record['path_txt_folder'])
    full_path_folder.mkdir(parents=True, exist_ok=True)

    full_path_article = Path(record['path_article_file'])
    full_path_txt = Path(record['path_txt_article'])

    if full_path_txt.exists() and full_path_txt.is_file():  
        continue

    if not(full_path_article.exists() and full_path_article.is_file()):
         continue

    if record['PDF']:

        doc = parser.from_file(str(full_path_article))['content']
        if not(doc):
             continue
       
    else:
        f = open(str(full_path_article),"r", encoding="utf-8")
        soup = BeautifulSoup(f.read(), "html.parser")
        f.close()
        doc = soup.get_text(separator='\n')
    
    with open(full_path_txt, "w", encoding="utf-8") as f:
        f.write(doc)


    

        


  0%|          | 0/5835 [00:00<?, ?it/s]